<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/07wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [2]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-wZj26ZHPIZ6kvcrPiL92q3&si=UF3qTgEXmUqnV4e->}}

# 2. Imports

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [3]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. CNN 핵심레이어 

## A. `torch.nn.ReLU`

## B. `torch.nn.MaxPool2d`

## C. `torch.nn.Conv2d`

**(예시1) 연산방법, stride=2**

In [4]:
img = torch.rand(1,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=2)

In [5]:
img

tensor([[[[0.7527, 0.4291, 0.5222, 0.7240],
          [0.8182, 0.3177, 0.2914, 0.2979],
          [0.5819, 0.4201, 0.1881, 0.9395],
          [0.7151, 0.2013, 0.1069, 0.2511]]]])

In [6]:
conv(img)

tensor([[[[0.1281, 0.0595],
          [0.1877, 0.0775]]]], grad_fn=<ConvolutionBackward0>)

??

In [7]:
conv.weight.data, conv.bias.data

(tensor([[[[-0.2970, -0.3377],
           [ 0.0756, -0.1169]]]]),
 tensor([0.4719]))

In [8]:
(img[:,  :,  :2,  :2] * conv.weight.data).sum()+conv.bias.data, conv(img)

(tensor([0.1281]),
 tensor([[[[0.1281, 0.0595],
           [0.1877, 0.0775]]]], grad_fn=<ConvolutionBackward0>))

In [9]:
(img[:,  :,  :2,  2:] * conv.weight.data).sum()+conv.bias.data, conv(img)

(tensor([0.0595]),
 tensor([[[[0.1281, 0.0595],
           [0.1877, 0.0775]]]], grad_fn=<ConvolutionBackward0>))

In [10]:
(img[:,  :,  :2,  2:] * conv.weight.data).sum()+conv.bias.data, conv(img)

(tensor([0.0595]),
 tensor([[[[0.1281, 0.0595],
           [0.1877, 0.0775]]]], grad_fn=<ConvolutionBackward0>))

In [11]:
(img[:,  :,  2:,  2:] * conv.weight.data).sum()+conv.bias.data, conv(img)

(tensor([0.0775]),
 tensor([[[[0.1281, 0.0595],
           [0.1877, 0.0775]]]], grad_fn=<ConvolutionBackward0>))

**(예시2) 이동평균**

In [13]:
img = torch.arange(1,17).float().reshape(1,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,bias=False)
conv.weight.data = conv.weight.data*0+1/4

In [14]:
img

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [15]:
conv(img)

tensor([[[[ 3.5000,  4.5000,  5.5000],
          [ 7.5000,  8.5000,  9.5000],
          [11.5000, 12.5000, 13.5000]]]], grad_fn=<ConvolutionBackward0>)

**(예시3) 2개의 이미지**

In [16]:
imgs = torch.arange(1,33).float().reshape(2,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,bias=False)
conv.weight.data = conv.weight.data*0+1/4

In [17]:
imgs

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]],


        [[[17., 18., 19., 20.],
          [21., 22., 23., 24.],
          [25., 26., 27., 28.],
          [29., 30., 31., 32.]]]])

In [18]:
conv(imgs)

tensor([[[[ 3.5000,  4.5000,  5.5000],
          [ 7.5000,  8.5000,  9.5000],
          [11.5000, 12.5000, 13.5000]]],


        [[[19.5000, 20.5000, 21.5000],
          [23.5000, 24.5000, 25.5000],
          [27.5000, 28.5000, 29.5000]]]], grad_fn=<ConvolutionBackward0>)

**(예시4) 2개의 out_channels**

In [19]:
img = torch.arange(1,33).float().reshape(2,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=2,kernel_size=2,stride=1,bias=False)
conv.weight.data[0] = conv.weight.data[0]*0 + 1/4
conv.weight.data[1] = conv.weight.data[1]*0 

In [20]:
img

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]],


        [[[17., 18., 19., 20.],
          [21., 22., 23., 24.],
          [25., 26., 27., 28.],
          [29., 30., 31., 32.]]]])

In [21]:
conv(img)

tensor([[[[ 3.5000,  4.5000,  5.5000],
          [ 7.5000,  8.5000,  9.5000],
          [11.5000, 12.5000, 13.5000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]],


        [[[19.5000, 20.5000, 21.5000],
          [23.5000, 24.5000, 25.5000],
          [27.5000, 28.5000, 29.5000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]]], grad_fn=<ConvolutionBackward0>)